<a href="https://colab.research.google.com/github/SomdeepAcharyya/Recommender-Systems/blob/main/TLRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TLRec:Transfer Learning for Cross-Domain Recommendation

## Data Reading

In [ ]:
import pandas as pd
import math
import numpy as np

In [ ]:
# MovieLens Dataset

path = r'/content/drive/MyDrive/Per_CD_RS/movielens_ratings.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  movielens = pd.read_csv(infile)
movielens = movielens[['userId', 'movieId', 'rating']]
movielens = movielens.rename(columns={"movieId": "itemId"})
movielens.columns

Index(['userId', 'itemId', 'rating'], dtype='object')

In [ ]:
# dividing the movielens dataset into 2 datasets 20%
source_user_ratings = movielens.sample(frac = 0.2)
target_user_ratings = movielens.drop(source_user_ratings.index)

In [ ]:
# creation of target rating matx for matx factorization(MF)
rs = source_user_ratings.pivot_table(index='userId',columns='itemId',values='rating')
print(rs.shape)
# creation of target rating matx for matx factorization(MF)
rt = target_user_ratings.pivot_table(index='userId',columns='itemId',values='rating')
print(rt.shape)

(668, 5362)
(668, 9526)


In [ ]:
rs

itemId,1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21,22,23,24,25,26,28,29,30,31,32,34,36,39,41,42,43,44,45,46,47,...,128975,129354,129428,129701,129937,130083,130520,130634,130970,130988,131013,131446,131656,131724,131842,132961,133195,133419,134130,134170,134368,134853,135133,135456,135460,135887,136020,138863,139385,139642,139644,140098,140110,140737,142074,142562,142566,143031,144482,146344
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Non Negative Matrix Factorization

In [ ]:
# creation of source rating matx for matx factorization(MF)
rs = source_user_ratings.pivot(index='userId',columns='itemId',values='rating')
rs.fillna(0, inplace=True)
rs.shape

(668, 5362)

In [ ]:
# creation of target rating matx for matx factorization(MF)
rt = target_user_ratings.pivot_table(index='userId',columns='itemId',values='rating')
rt.fillna(0, inplace=True)
rt.shape

(668, 9526)

In [ ]:
X_s = np.array(rs)
X_t = np.array(rt)

In [ ]:
from sklearn.decomposition import NMF
model_s = NMF(n_components=2, init='random', random_state=0)
W_s= model_s.fit_transform(X_s)
H_s = model_s.components_
model_t = NMF(n_components=2, init='random', random_state=0)
W_t= model_t.fit_transform(X_t)
H_t = model_t.components_

In [ ]:
us = W_s
vs = H_s
ut = W_t
vt = H_t

In [ ]:
us

In [ ]:
print(us.shape, vs.shape)
print(ut.shape, vt.shape)

(668, 2) (2, 5362)
(668, 2) (2, 9526)


## Problem formulation

In [ ]:
# check UtV inner product
uv_s = np.matmul(us, vs)
uv_t = np.matmul(ut, vt)
print(uv_s)
print(uv_t)

[[3.23984828e-01 1.58224607e-01 1.27083709e-01 ... 3.42410689e-03
  1.75077066e-02 8.66098920e-03]
 [4.26551596e-02 1.65712246e-02 1.34835519e-02 ... 3.73930356e-04
  1.93452867e-03 9.78147560e-04]
 [1.28341803e-01 6.25346823e-02 5.02328098e-02 ... 1.35381693e-03
  6.92292600e-03 3.42545551e-03]
 ...
 [2.17805486e-01 1.06369757e-01 8.54346460e-02 ... 2.30192652e-03
  1.17699171e-02 5.82252872e-03]
 [1.06869346e-01 5.21918274e-02 4.19197187e-02 ... 1.12947284e-03
  5.77507645e-03 2.85690615e-03]
 [3.09759429e+00 0.00000000e+00 6.17086329e-02 ... 5.43844260e-03
  3.58305697e-02 2.52333002e-02]]
[[1.29097520e+00 5.45348681e-01 1.58661484e-01 ... 4.55659988e-03
  0.00000000e+00 6.07546651e-03]
 [2.58028196e-01 1.08999256e-01 3.17117917e-02 ... 9.10731088e-04
  0.00000000e+00 1.21430812e-03]
 [6.88660731e-01 2.90912035e-01 8.46367413e-02 ... 2.43068296e-03
  0.00000000e+00 3.24091061e-03]
 ...
 [1.26492271e+00 5.39922679e-01 1.60717615e-01 ... 5.58724921e-03
  2.03058845e-02 7.44966562e-03]

In [ ]:
# create indicator matrix (mask matrix)
rs_m = rs > 0
rt_m = rt > 0
rs_m = rs_m.replace(True, 1)
rs_m = rs_m.replace(False, 0)
rt_m = rt_m.replace(True, 1)
rt_m = rt_m.replace(False, 0)
rs_m

itemId,1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21,22,23,24,25,26,28,29,30,31,32,34,36,39,41,42,43,44,45,46,47,...,128975,129354,129428,129701,129937,130083,130520,130634,130970,130988,131013,131446,131656,131724,131842,132961,133195,133419,134130,134170,134368,134853,135133,135456,135460,135887,136020,138863,139385,139642,139644,140098,140110,140737,142074,142562,142566,143031,144482,146344
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
665,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Equation 1 of L1
L1 = 0
IR_s = np.array(rs_m)
t = np.multiply(IR_s, math.pow(np.subtract(np.array(rs), np.array(uv_s))), 2)
print(t.shape)
t = np.sum(t, axis=1)  # column sum items j
print(t.shape)
t = np.sum(t, axis=0)  # row sum users i
L1 = 1/2*t
L1

(668, 5362)
(668,)


33579.98591316006

In [ ]:
# get similar users Nu
from scipy.spatial import distance
su = movielens.userId.drop_duplicates()
su.value_counts()
t = movielens.pivot_table(index='userId',columns='itemId',values='rating')
t.fillna(0, inplace=True)
t = np.array(t)
k = 10
ud = []
for a in range(len(su)):
  rows = []
  for b in range(k):
    rows.append(0)
  ud.append(rows)
su = pd.DataFrame()
su['userId'] = movielens.userId.drop_duplicates()
su['dist'] = ud
su['similar_userId'] = ud
su['similar_user_latent_factor'] = ud
su = su.reset_index()
for i in range(len(su)):
    user = su['userId'][i]
    x = (pd.DataFrame())
    x['dist'] = 0.00
    x['id'] = 0
    x['sim'] = 0.00
    c = []
    e = []
    f = []
    for j in range(len(t)):
      c.append(distance.euclidean(t[i], t[j]))
      e.append(j)
      f.append(t[j])
    x['dist'] = c
    x['id'] = e
    x['sim'] = f
    x = x.sort_values(by='dist')
    y = x[0:k]
    su['dist'][i] = np.array(y['dist'])
    su['similar_userId'][i] =  np.array(y['id'])
    su['similar_user_latent_factor'][i] =  np.array(y['sim'])

Nu = su
print(Nu)

In [ ]:
# get similar items Nv
from scipy.spatial import distance
su = movielens.itemId.drop_duplicates()
su.value_counts()
t = movielens.pivot_table(index='itemId',columns='userId',values='rating')
t.fillna(0, inplace=True)
t = np.array(t)
k = 10
ud = []
for a in range(len(su)):
  rows = []
  for b in range(k):
    rows.append(0)
  ud.append(rows)
su = pd.DataFrame()
su['itemId'] = movielens.itemId.drop_duplicates()
su['dist'] = ud
su['similar_itemId'] = ud
su['similar_item_latent_factor'] = ud
su = su.reset_index()
for i in range(len(su)):
    item = su['itemId'][i]
    x = (pd.DataFrame())
    x['dist'] = 0.00
    x['id'] = 0
    x['sim'] = 0.00
    c = []
    e = []
    f = []
    for j in range(len(t)):
      c.append(distance.euclidean(t[i], t[j]))
      e.append(j)
      f.append(t[j])
    x['dist'] = c
    x['id'] = e
    x['sim'] = f
    x = x.sort_values(by='dist')
    y = x[0:k]
    su['dist'][i] = np.array(y['dist'])
    su['similar_itemId'][i] =  np.array(y['id'])
    su['similar_item_latent_factor'][i] =  np.array(y['sim'])

Nv = su
print(Nv)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
users = movielens.userId.drop_duplicates()
su = []
t = movielens.pivot_table(index='userId',columns='itemId',values='rating')
t.fillna(0, inplace=True)
t = np.array(t)
for i in range(len(t)):
  rows = []
  for j in range(len(t)):
    rows.append(distance.euclidean(t[i], t[j]))
  su.append(rows)

print(su)

items = movielens.userId.drop_duplicates()
sv = []
t = movielens.pivot_table(index='itemId',columns='userId',values='rating')
t.fillna(0, inplace=True)
t = np.array(t)
for i in range(len(t)):
  rows = []
  for j in range(len(t)):
    rows.append(distance.euclidean(t[i], t[j]))
  sv.append(rows)

print(sv)

du = []
for i in range(len(Nu)):
  row = []
  for j in range(len(Nu['similar_userId'][i])):
    x = Nu['similar_userId'][i][j]
    row.append(su[i][x])
  du.append(row)

dv = []
for i in range(len(Nv)):
  row = []
  for j in range(len(Nv['similar_itemId'][i])):
    x = Nu['similar_itemId'][i][j]
    row.append(sv[i][x])
  dv.append(row)

di =  np.sum(du, axis=1)
dj =  np.sum(dv, axis=1)

In [ ]:
# Equation 2 of L2 and L3
b1 = 5, b2= 1, n1=0.01, n2= 0.01  # smoothness and regulariser parameters
L2 = 0, L3 = 0
x = []
y = []
for i in range(len(us)):
  x.append(math.pow(np.linalg.norm(us[i]), 2))
  row = []
  for h in range(len(Nu)):
    row.append(su[i][Nu[h]]/di[i] * math.pow(np.linalg.norm(np.subtract(us[i]-us[Nu[h]])), 2))
  y.append(np.sum(row, axis=1))

x = np.sum(x, axis=0)
y = np.sum(y, axis=0)
L2 = b1*x/2+n1*y/2  
x = []
y = []
for j in range(len(vs)):
  x.append(math.pow(np.linalg.norm(vs[j]), 2))
  row = []
  for h in range(len(Nv)):
    row.append(sv[j][Nv[h]]/dj[j] * math.pow(np.linalg.norm(np.subtract(vs[j]-vs[Nv[h]])), 2))
  y.append(np.sum(row, axis=1))

x = np.sum(x, axis=0)
y = np.sum(y, axis=0)
L3 = b2*x/2+n2*y/2   

print(L2, L3)

(631,)

In [ ]:
# Loss function L Eq 3 L(U, V)
L = L1+L2+L3

array([[0.00000000e+00, 3.40551175e-01],
       [2.22398515e-02, 3.56667026e-02],
       [7.49932605e-04, 1.34595117e-01],
       ...,
       [0.00000000e+00, 2.28942555e-01],
       [0.00000000e+00, 1.12333906e-01],
       [7.89707442e+00, 0.00000000e+00]])

In [ ]:
# stochastic gradient descent (SGD) to learn the user and item latent vectors from the objective loss function in Eq 3
# compute the associated prdection error E = I(R-UV)
# modify the parameters by learning rate lambda in the opposite direction of gradient
# given eq 4


In [ ]:
# use of Vector space similarity, Pearson Correlation Coefficient, Jaccard Coeffiecients and SimRank as similarity measure between user i and item j